### Athena + Polars ###

- query distribuída via Athena

- análise local via Polars

In [ ]:
# Query Athena

import boto3

athena = boto3.client("athena")

QUERY = """
SELECT movieId, AVG(rating) AS avg_rating
FROM ratings_32m
GROUP BY movieId
"""

OUTPUT = "s3://meu-bucket/athena-results/"




In [ ]:
# Executar query + medir tempo

start = time.time()

resp = athena.start_query_execution(
    QueryString=QUERY,
    QueryExecutionContext={"Database": "movielens"},
    ResultConfiguration={"OutputLocation": OUTPUT}
)

query_id = resp["QueryExecutionId"]

status = "RUNNING"
while status in ("RUNNING", "QUEUED"):
    status = athena.get_query_execution(QueryExecutionId=query_id)["QueryExecution"]["Status"]["State"]

athena_time = round(time.time() - start, 2)
athena_time


In [ ]:
# Ler resultado com Polars

result_path = f"{OUTPUT}{query_id}.csv"
df_athena = pl.read_csv(result_path)
df_athena.head()
